###### May 2022
###### Author: Martina Lorenzo Valerio 

# Data Visualisation: Amazon product co-purchasing network metadata

#### Import libraries

In [5]:
import string
import re
from nltk.corpus import stopwords
from stemming.porter2 import stem
import networkx

In [6]:
#open the file to read Amazon product metadata from http://snap.stanford.edu/data/amazon-meta.html

fhr = open ('/Users/martinamanno/Desktop/LUISS/CORSI da seguire/DATA VISUALIZATION/project/amazon-meta.txt', 'r', encoding='utf-8', errors= 'ignore')

#### Description of the dataset
The dataset provides product and review metadata on 548,552 different products. The data was collected in 2006 by crawling the Amazon website. The dataset contains information about:
- ID:product number
- ASIN: Amazon Standard Identification Number is a 10-character alphanumeric unique identifier assigned by Amazon.com for product identification
- Title: title of the product
- Group: the product groups can be Book, DVD, Video or Music
- Sales rank: the Amazon sales rank represents how a product is selling in comparison to other products in its primary category. The lower the rank, the better a product is selling
- Similar: ASINs of co-purchased products, such as people who buy X also buy Y
- Categories: location in product category hierarchy to which the product belongs (separated by |, category id in [])
- Reviews: product review information such as the total number of reviews, average rating and the individual customer review information including time, user id, rating, total number of votes on the review, total number of helpfulness votes which means how many people found the review to be helpful


 

#### Dataset preprocessing

Initialize a nested product dictionary that will hold cleaned up amazon product data. 



In [7]:
amazonProducts= {}

Before analyzing the social network, some processes are required to use ASIN as the key and the others as the metadata associated with ASIN. 
- All the categories associated with the ASIN are concatenated, and then the are subject to Text Preprocessing steps: lowercase, stemming, remove digit/punctuation, remove stop words, retain only unique words. 
- The copurchased ASINs in the “similar” field are filtered down to only those ASINs that have metadata associated with it.

In [8]:
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)

for line in fhr:
    line = line.strip()
    # a pduct block started
    if(line.startswith("Id")):
        Id = line[3:].strip()
    elif(line.startswith("ASIN")):
        ASIN = line[5:].strip()
    elif(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
        #Titlefidf = Title.tolower()
        #Titlefidf = re.compile('[%s]' % re.escape(string.digit+string.punctuation)).sub(' ',Titlefidf)
        #Titlefidf = ' '.join(set(Titlefidf.split())-set(stopwords.words("english")))
        #Titlefidf = ' '.join(stem(word) for word in Titlefidf.split())
    elif(line.startswith("group")):
        Group = line[6:].strip()
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ' '.join([c for c in ls[2:]])
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
    # a product block eneded
    # write out fields to amazonProducts Dictionary
    elif (line==""):
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN] = MetaData
            MetaData['Id'] = Id
            MetaData['Title'] = Title
            MetaData['Categories'] = ' '.join(set(Categories.split()))
            MetaData['Group'] = Group
            MetaData['Copurchased'] = Copurchased
            MetaData['SalesRank'] = int(SalesRank)
            MetaData['TotalReviews'] = int(TotalReviews)
            MetaData['AvgRating'] = float(AvgRating)
            MetaData['DegreeCentrality'] = DegreeCentrality
            MetaData['ClusteringCoeff'] = ClusteringCoeff
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
fhr.close
        

<function TextIOWrapper.close()>

Filter Amazon products dictionary down to only Group=Book and write it to Amazon books dictionary.

In [10]:
#create book specific dictionary exclusively for books
amazonBooks = {}
for asin,metadata in amazonProducts.items():
    if (metadata['Group']=='Book'):
        amazonBooks[asin]=amazonProducts[asin]
        
#remove any copurchased items from copurchase list. If we don't have metadata associated with it
for asin, metadata in amazonBooks.items():
    amazonBooks[asin]['Copurchased']= ' '.join([cp for cp in metadata['Copurchased'].split() if cp in amazonBooks.keys()])
    

Use the copurchase data in Amazon books dictionary to create the copurchase graph structure as follows:
- NODES: are the ASINs
- EDGES: exist between two Nodes if the two ASINs were co-purchased
- EDGE WEIGHT: it is based on category similarity
- SIMILARITY: it is a measure between any two ASINs that were co-purchased and it can be calculated as the number of words that are common between categories of connected Nodes divided by the total number of words in both categories of connected nodes. The similarity range goes from 0 (most dissimilar) to 1 (most similar).

 

In [11]:
#create a product copurchase graph for analysis
#the graph nodes are product ASINs, the graph edge exists if two products were copurchased, with edge weight being a measure of category similarity between ASINs
copurchaseGraph = networkx.Graph()
for asin, metadata in amazonBooks.items():
    copurchaseGraph.add_node(asin)
    for a in metadata ['Copurchased'].split():
        copurchaseGraph.add_node(a.strip())
        similarity=0
        n1= set((amazonBooks[asin]['Categories']).split())
        n2= set ((amazonBooks[a]['Categories']).split())
        n1In2 = n1 & n2
        n1Un2 = n1 | n2
        if (len(n1Un2)) > 0:
            similarity = round (len(n1In2)/len(n1Un2), 2)
            copurchaseGraph.add_edge(asin, a.strip(), weight=similarity)

Add the graph-related measures for each ASIN node to the Amazon books dictionary: degree centrality and clustering coefficient.

In [12]:
dc = networkx.degree(copurchaseGraph)
for asin in networkx.nodes(copurchaseGraph):
    metadata = amazonBooks[asin]
    metadata['DegreeCentrality'] = int(dc[asin])
    ego = networkx.ego_graph(copurchaseGraph, asin, radius = 1)
    metadata['ClusteringCoeff'] = round(networkx.average_clustering(ego), 2)
    amazonBooks[asin] = metadata

Now, write out Amazon books data to the amazon-books.txt file and the copurchase graph data to the amazon-books-copurchase.edgelist file.


In [13]:
fhw = open('./amazon-books.txt', 'w', encoding = 'utf-8', errors = 'ignore')
fhw.write('Id\t' + 'ASIN\t' + 'Title\t'+
         'Categories\t' + 'Group\t' + 'Copurchased\t'+
         'SalesRank\t' + 'TotalReviews\t' + 'AvgRating\t'+
         'DegreeCentrality\t' + 'ClusteringCoeff\n')
for asin, metadata in amazonBooks.items():
    fhw.write(metadata['Id'] + '\t' + 
              asin + '\t' +
              metadata['Title'] + '\t' +
              metadata['Categories'] + '\t' +
              metadata['Group'] + '\t' +
              metadata['Copurchased'] +'\t' + 
              str(metadata['SalesRank']) + '\t' +
              str(metadata['TotalReviews']) + '\t' +
              str(metadata['AvgRating']) + '\t' +
              str(metadata['DegreeCentrality']) + '\t' +
              str(metadata['ClusteringCoeff']) + '\n')
fhw.close()

# write copurchaseGraph to file
fhw = open('amazon-books-copurchase.edgelist', 'wb')
networkx.write_weighted_edgelist(copurchaseGraph, fhw)
fhw.close()